# Config Servers – Setup

This notebook deploys **only the config servers** (`configsvr` role) that form the
`config_rs` replica set.

In [63]:
MONGODB_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.30.1"]

# --- CONFIG SERVER CONFIGURATION ---
MONGODB_CONFIG_SVR_NODES = 3
MONGODB_STARTING_PORT = 27010
# ------------------------------------

MONGO_INITDB_ROOT_USERNAME = "admin"
MONGO_INITDB_ROOT_PASSWORD = "admin"
MONGO_INITDB_DATABASE = "admin"

# --- DYNAMIC NAME & PORT GENERATION ---
MONGODB_CONFIG_SVR_NAMES = [
    f"config-server-{i+1}" for i in range(MONGODB_CONFIG_SVR_NODES)
]
MONGODB_CONFIG_SVR_PORTS = [
    MONGODB_STARTING_PORT + (i + 1) for i in range(MONGODB_CONFIG_SVR_NODES)
]
MONGODB_CONFIG_SVR_HOSTNAMES = [
    f"{name}.mavasbel.vpn.itam.mx" for name in MONGODB_CONFIG_SVR_NAMES
]

print(
    "Config servers:",
    *list(
        zip(
            MONGODB_CONFIG_SVR_NAMES,
            MONGODB_CONFIG_SVR_HOSTNAMES,
            MONGODB_CONFIG_SVR_PORTS,
        )
    ),
    sep="\n",
)

Config servers:
('config-server-1', 'config-server-1.mavasbel.vpn.itam.mx', 27011)
('config-server-2', 'config-server-2.mavasbel.vpn.itam.mx', 27012)
('config-server-3', 'config-server-3.mavasbel.vpn.itam.mx', 27013)


In [48]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
LOCALHOST_CONFIG_DIR = os.path.abspath("config")
LOCALHOST_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
MONGODB_LOCAL_CLUSTER_KEY_PATH = os.path.join(LOCALHOST_WORKDIR, "mongo-keyfile")

mount_path = Path(LOCALHOST_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop mongodb-configsvr.docker-compose.yml

In [49]:
!docker compose -f mongodb-configsvr.docker-compose.yml down -v

 Container config-server-3  Stopping
 Container config-server-3  Stopped
 Container config-server-3  Removing
 Container config-server-3  Removed
 Container config-server-2  Stopping
 Container config-server-2  Stopped
 Container config-server-2  Removing
 Container config-server-2  Removed
 Container config-server-1  Stopping
 Container config-server-1  Stopped
 Container config-server-1  Removing
 Container config-server-1  Removed
 Network mongodb-configsvr_mongodb-network  Removing
 Network mongodb-configsvr_mongodb-network  Removed


In [50]:
import shutil
import stat

if MONGODB_START_FROM_SCRATCH:
    if os.path.exists(MONGODB_LOCAL_CLUSTER_KEY_PATH):
        os.chmod(MONGODB_LOCAL_CLUSTER_KEY_PATH, stat.S_IWRITE)
        os.remove(MONGODB_LOCAL_CLUSTER_KEY_PATH)
    if os.path.exists(LOCALHOST_CONFIG_DIR):
        shutil.rmtree(LOCALHOST_CONFIG_DIR)
    if os.path.exists(LOCALHOST_MOUNTDIR):
        shutil.rmtree(LOCALHOST_MOUNTDIR)

# Start mongodb-configsvr.docker-compose.yml

In [51]:
import yaml
import base64
import secrets
from IPython.display import Markdown, display

# --- Generate keyfile if it doesn't exist ---
if not os.path.exists(MONGODB_LOCAL_CLUSTER_KEY_PATH):
    with open(MONGODB_LOCAL_CLUSTER_KEY_PATH, "w") as f:
        raw_data = secrets.token_bytes(756)
        f.write(base64.b64encode(raw_data).decode("utf-8"))

In [52]:
# --- Generate mongod.conf and mongod-bootstrap.sh for each config server ---
#
# mongod-bootstrap.sh implements the two-phase startup that docker-entrypoint.sh
# cannot handle for configsvr nodes:
#   Phase 1 – plain standalone (no configsvr/keyFile/replSet) to create the admin user
#   Phase 2 – exec real mongod with the full config

MONGO_REPLICA_SET_NAME = "config_rs"
MONGO_BOOTSTRAP_SCRIPT = """\
#!/bin/bash
set -Eeuo pipefail

# --- Step 1: Secure the keyfile ---
cp -f /etc/mongo/config/mongo-keyfile /data/keyfile
chmod 400 /data/keyfile
chown 999:999 /data/keyfile

# --- Step 2: Skip init if already initialized ---
for marker in /data/db/WiredTiger /data/db/storage.bson /data/db/journal; do
    if [ -e "$marker" ]; then
        echo "[bootstrap] Data directory already initialized – skipping user creation."
        exec mongod --config /etc/mongo/config/mongod.conf
    fi
done

# --- Step 3: Phase 1 – plain standalone, no configsvr/keyFile/replSet ---
echo "[bootstrap] Starting temporary standalone mongod for user initialisation..."
mongod \\
    --dbpath /data/db \\
    --bind_ip 127.0.0.1 \\
    --port 27017 \\
    --fork \\
    --logpath /tmp/mongod-init.log

echo "[bootstrap] Waiting for temporary mongod to accept connections..."
until mongosh --quiet --eval "db.adminCommand('ping')" > /dev/null 2>&1; do
    sleep 1
done

echo "[bootstrap] Creating admin user..."
mongosh admin --quiet --eval "
db.createUser({
    user: '$MONGO_INITDB_ROOT_USERNAME',
    pwd:  '$MONGO_INITDB_ROOT_PASSWORD',
    roles: [{ role: 'root', db: '$MONGO_INITDB_DATABASE' }]
});
"

echo "[bootstrap] Shutting down temporary mongod..."
mongod --dbpath /data/db --shutdown

echo "[bootstrap] Init complete – starting production mongod."

# --- Step 4: Phase 2 – real mongod with keyFile, configsvr, replSet ---
exec mongod --config /etc/mongo/config/mongod.conf
"""

os.makedirs(LOCALHOST_CONFIG_DIR, exist_ok=True)
for i, node_name in enumerate(MONGODB_CONFIG_SVR_NAMES):
    node_config_dir = os.path.join(LOCALHOST_CONFIG_DIR, node_name)
    os.makedirs(node_config_dir, exist_ok=True)

    # Keyfile
    shutil.copy(
        MONGODB_LOCAL_CLUSTER_KEY_PATH, os.path.join(node_config_dir, "mongo-keyfile")
    )

    # mongod.conf — storage.dbPath explicit so bootstrap finds the right directory
    config_dict = {
        "storage": {"dbPath": "/data/db"},
        "net": {"bindIp": "0.0.0.0", "port": MONGODB_CONFIG_SVR_PORTS[i]},
        "security": {"keyFile": "/data/keyfile"},
        "sharding": {"clusterRole": "configsvr"},
        "replication": {"replSetName": MONGO_REPLICA_SET_NAME},
    }
    with open(
        os.path.join(node_config_dir, "mongod.conf"),
        "w",
        encoding="utf-8",
        newline="\n",
    ) as f:
        yaml.dump(config_dict, f, default_flow_style=False)

    # mongod-bootstrap.sh
    bootstrap_path = os.path.join(node_config_dir, "mongod-bootstrap.sh")
    with open(bootstrap_path, "w", encoding="utf-8", newline="\n") as f:
        f.write(MONGO_BOOTSTRAP_SCRIPT)

print("✅ Configuration files generated in 'config/' directory.")

✅ Configuration files generated in 'config/' directory.


In [53]:
# --- Copy config files into mount directories ---
for node_name in MONGODB_CONFIG_SVR_NAMES:
    src_config_dir = os.path.join(LOCALHOST_CONFIG_DIR, node_name)
    dest_config_dir = os.path.join(LOCALHOST_MOUNTDIR, node_name, "config")
    dest_data_dir = os.path.join(LOCALHOST_MOUNTDIR, node_name, "data")

    if MONGODB_START_FROM_SCRATCH and os.path.exists(dest_config_dir):
        shutil.rmtree(dest_config_dir)
    if MONGODB_START_FROM_SCRATCH and os.path.exists(dest_data_dir):
        shutil.rmtree(dest_data_dir)
    os.makedirs(dest_config_dir, exist_ok=True)
    os.makedirs(dest_data_dir, exist_ok=True)

    shutil.copytree(src_config_dir, dest_config_dir, dirs_exist_ok=True)

print("✅ Configuration copied to mount directories.")

✅ Configuration copied to mount directories.


In [54]:
# --- Generate Docker Compose file ---
configsvr_compose_dict = {
    "name": "mongodb-configsvr",
    "services": {},
    "networks": {"mongodb-network": {"driver": "bridge"}},
}

# The bootstrap script handles keyfile copy + two-phase init + real mongod exec.
# MONGO_INITDB_ROOT_* env vars are read directly by the script via shell expansion.
cmd_script = "bash /etc/mongo/config/mongod-bootstrap.sh"

for i, node_name in enumerate(MONGODB_CONFIG_SVR_NAMES):
    hostname = MONGODB_CONFIG_SVR_HOSTNAMES[i]
    port = MONGODB_CONFIG_SVR_PORTS[i]

    service_def = {
        "image": "mongo:7.0",
        "container_name": node_name,
        "hostname": hostname,
        "command": ["bash", "-c", cmd_script],
        "environment": [
            f"MONGO_INITDB_ROOT_USERNAME={MONGO_INITDB_ROOT_USERNAME}",
            f"MONGO_INITDB_ROOT_PASSWORD={MONGO_INITDB_ROOT_PASSWORD}",
            f"MONGO_INITDB_DATABASE={MONGO_INITDB_DATABASE}",
        ],
        "volumes": [
            f"{os.path.join(LOCALHOST_MOUNTDIR, node_name, 'data')}:/data/db",
            f"{os.path.join(LOCALHOST_MOUNTDIR, node_name, 'config')}:/etc/mongo/config",
        ],
        "networks": ["mongodb-network"],
        "ports": [f"{port}:{port}"],
        "extra_hosts": [f"{DOCKER_INTERNAL_HOST}:host-gateway"],
        "dns": DOCKER_DNS,
        "deploy": {"resources": {"limits": {"cpus": "1.0", "memory": "512M"}}},
        "healthcheck": {
            "test": [
                "CMD",
                "mongosh",
                "--port",
                f"{port}",
                "--quiet",
                "--eval",
                "db.adminCommand('ping')",
            ],
            "interval": "10s",
            "timeout": "10s",
            "retries": 10,
            "start_period": "30s",  # allow time for the two-phase init
        },
        "depends_on": {},
    }

    # Each server depends on the previous one having started
    if i > 0:
        service_def["depends_on"][MONGODB_CONFIG_SVR_NAMES[i - 1]] = {
            "condition": "service_started"
        }

    configsvr_compose_dict["services"][node_name] = service_def

compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "mongodb-configsvr.docker-compose.yml"
)
with open(compose_yaml_path, "w") as f:
    yaml.dump(
        configsvr_compose_dict, f, default_flow_style=False, sort_keys=False, indent=4
    )

print(f"Successfully created: '{os.path.relpath(compose_yaml_path)}'")
display(
    Markdown(
        f"```yaml\n{yaml.dump(configsvr_compose_dict, default_flow_style=False, sort_keys=False, indent=4)}\n```"
    )
)

Successfully created: 'mongodb-configsvr.docker-compose.yml'


```yaml
name: mongodb-configsvr
services:
    config-server-1:
        image: mongo:7.0
        container_name: config-server-1
        hostname: config-server-1.mavasbel.vpn.itam.mx
        command:
        - bash
        - -c
        - bash /etc/mongo/config/mongod-bootstrap.sh
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\config-server-1\data:/data/db
        - .\mount\config-server-1\config:/etc/mongo/config
        networks:
        - mongodb-network
        ports:
        - 27011:27011
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: &id001
        - 10.15.30.1
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 512M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27011'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
        depends_on: {}
    config-server-2:
        image: mongo:7.0
        container_name: config-server-2
        hostname: config-server-2.mavasbel.vpn.itam.mx
        command:
        - bash
        - -c
        - bash /etc/mongo/config/mongod-bootstrap.sh
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\config-server-2\data:/data/db
        - .\mount\config-server-2\config:/etc/mongo/config
        networks:
        - mongodb-network
        ports:
        - 27012:27012
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 512M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27012'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
        depends_on:
            config-server-1:
                condition: service_started
    config-server-3:
        image: mongo:7.0
        container_name: config-server-3
        hostname: config-server-3.mavasbel.vpn.itam.mx
        command:
        - bash
        - -c
        - bash /etc/mongo/config/mongod-bootstrap.sh
        environment:
        - MONGO_INITDB_ROOT_USERNAME=admin
        - MONGO_INITDB_ROOT_PASSWORD=admin
        - MONGO_INITDB_DATABASE=admin
        volumes:
        - .\mount\config-server-3\data:/data/db
        - .\mount\config-server-3\config:/etc/mongo/config
        networks:
        - mongodb-network
        ports:
        - 27013:27013
        extra_hosts:
        - host.docker.internal:host-gateway
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '1.0'
                    memory: 512M
        healthcheck:
            test:
            - CMD
            - mongosh
            - --port
            - '27013'
            - --quiet
            - --eval
            - db.adminCommand('ping')
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 30s
        depends_on:
            config-server-2:
                condition: service_started
networks:
    mongodb-network:
        driver: bridge

```

In [55]:
!docker compose -f mongodb-configsvr.docker-compose.yml up -d --wait

 Network mongodb-configsvr_mongodb-network  Creating
 Network mongodb-configsvr_mongodb-network  Created
 Container config-server-1  Creating
 Container config-server-1  Created
 Container config-server-2  Creating
 Container config-server-2  Created
 Container config-server-3  Creating
 Container config-server-3  Created
 Container config-server-1  Starting
 Container config-server-1  Started
 Container config-server-2  Starting
 Container config-server-2  Started
 Container config-server-3  Starting
 Container config-server-3  Started
 Container config-server-2  Waiting
 Container config-server-3  Waiting
 Container config-server-1  Waiting
 Container config-server-1  Healthy
 Container config-server-2  Healthy
 Container config-server-3  Healthy


# Initialise the `config_rs` Replica Set

At this point the containers are healthy. The bootstrap script has created the
admin user during Phase 1, so we can connect with credentials.

We connect to `localhost` on each published port but list the full Docker-internal
hostnames in the `replSetInitiate` member config so inter-container replication
works correctly inside the Docker network.

In [56]:
import time
from pymongo import MongoClient
from pymongo.errors import OperationFailure

client_options = {"directConnection": True, "serverSelectionTimeoutMS": 20000}

auth_uri = (
    f"mongodb://{MONGO_INITDB_ROOT_USERNAME}:{MONGO_INITDB_ROOT_PASSWORD}"
    f"@{MONGODB_CONFIG_SVR_HOSTNAMES[0]}:{MONGODB_CONFIG_SVR_PORTS[0]}/?authSource=admin"
)

replica_set_config = {
    "_id": MONGO_REPLICA_SET_NAME,
    "configsvr": True,
    "members": [
        {
            "_id": i,
            "host": f"{MONGODB_CONFIG_SVR_HOSTNAMES[i]}:{MONGODB_CONFIG_SVR_PORTS[i]}",
        }
        for i in range(MONGODB_CONFIG_SVR_NODES)
    ],
}

print(
    f"🚀 Initiating replica set '{MONGO_REPLICA_SET_NAME}' on localhost:{MONGODB_CONFIG_SVR_PORTS[0]} ..."
)
for m in replica_set_config["members"]:
    print(f"   _id={m['_id']}  host={m['host']}")

try:
    with MongoClient(auth_uri, **client_options) as client:
        client.admin.command("replSetInitiate", replica_set_config)
        print("✅ replSetInitiate accepted.")
except OperationFailure as e:
    if "already initialized" in str(e).lower():
        print(
            f"⚠️  Replica set '{MONGO_REPLICA_SET_NAME}' is already initiated. Skipping..."
        )
    else:
        raise

🚀 Initiating replica set 'config_rs' on localhost:27011 ...
   _id=0  host=config-server-1.mavasbel.vpn.itam.mx:27011
   _id=1  host=config-server-2.mavasbel.vpn.itam.mx:27012
   _id=2  host=config-server-3.mavasbel.vpn.itam.mx:27013
✅ replSetInitiate accepted.


In [64]:
import time

PRIMARY_ELECTION_TIMEOUT = 30

primary = None
start_time = time.time()
with MongoClient(auth_uri, **client_options) as client:
    while primary is None and time.time() < start_time + PRIMARY_ELECTION_TIMEOUT:
        status = client.admin.command("replSetGetStatus")
        primary = next(
            (m for m in status.get("members", []) if m["stateStr"] == "PRIMARY"),
            None,
        )
        if primary is None:
            print("⏳ No PRIMARY yet, retrying in 1 s...")
            time.sleep(1)
        else:
            print(f"Replica Set '{MONGO_REPLICA_SET_NAME}' Status Summary:")
            for m in status["members"]:
                icon = "🟢" if m["health"] == 1 else "🔴"
                print(f"{icon} {m['name']:<35} | {m['stateStr']:<10}")

if primary is None:
    raise TimeoutError(f"No PRIMARY elected within {PRIMARY_ELECTION_TIMEOUT}s.")

Replica Set 'config_rs' Status Summary:
🟢 config-server-1.mavasbel.vpn.itam.mx:27011 | PRIMARY   
🟢 config-server-2.mavasbel.vpn.itam.mx:27012 | SECONDARY 
🟢 config-server-3.mavasbel.vpn.itam.mx:27013 | SECONDARY 
